# General Settings
---

## Import Libs
---

In [ ]:
# System Libs
import multiprocessing as mp
import sys
import os
import argparse
import json
from importlib import import_module
from pathlib import Path
from glob import glob
from time import time

# Other Libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.metrics import f1_score
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import f1_score
import wandb

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import models
from torchvision import transforms
from torchvision.transforms import CenterCrop, Resize, ToTensor, Normalize
device = ("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Local Libs
from dataset import TrainInfo, MaskBaseDataset, TestDataset
from model import BaseModel, ResNet18Pretrained
from loss import get_criterion
import settings
import logger

# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# Argument
parser = argparse.ArgumentParser()

parser.add_argument('--transform', type=str, default=('BaseTransform', 'BaseTransform'), help='data transform type (default: ("BaseTransform", "CustomTransform"))')
parser.add_argument("--resize", nargs="+", type=list, default=(300, 200), help='resize size for image when training (default: (128, 96))')

parser.add_argument('--data_dir', type=str, default=os.environ.get('SM_CHANNEL_TRAIN', '/opt/ml/input/data/train/images'))
parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR', './model'))
parser.add_argument('--seed', type=int, default=42, help='random seed (default: 42)')
parser.add_argument('--epochs', type=int, default=10, help='number of epochs to train (default: 20)')
parser.add_argument('--dataset', type=str, default='MaskBaseDataset', help='dataset transform type (default: MaskBaseDataset)')
parser.add_argument('--batch_size', type=int, default=64, help='input batch size for training (default: 128)')
parser.add_argument('--val_batch_size', type=int, default=10, help='input batch size for validation (default: 1000)')
parser.add_argument('--model', type=str, default='ResNet18Pretrained', help='model type (default: ResNet18Pretrained)')
parser.add_argument('--optimizer', type=str, default='Adam', help='optimizer type (default: Adam)')
parser.add_argument('--lr', type=float, default=1e-3, help='learning rate (default: 1e-4)')
parser.add_argument('--val_ratio', type=float, default=0.2, help='ratio for validaton (default: 0.2)')
parser.add_argument('--criterion', type=str, default='cross_entropy', help='criterion type (default: cross_entropy)')
parser.add_argument('--lr_decay_step', type=int, default=20, help='learning rate scheduler deacy step (default: 20)')
parser.add_argument('--log_interval', type=int, default=10, help='how many batches to wait before logging training status')
parser.add_argument('--name', type=str, default='exp_kh', help='model to save at {SM_MODEL_DIR}/{name}')
parser.add_argument('--mode', type=str, default='', help='select mask, age, gender, ensemble')
parser.add_argument('--model_name', type=str, default='best', help='custom model name')
parser.add_argument('--freeze', nargs='+', default=['layer1', 'layer2'], help='layers to freeze (default: [])')



parser.add_argument('--dump', type=bool, default=False, help="choose dump or not to save model")
args = parser.parse_args([])

helper = settings.SettingsHelper(
    args=args,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)

with open('wandb_config.json', 'r') as f:
    wb_object = json.load(f)
    project, entity, name = wb_object['init'].values()
    wandb.init(project=project, entity=entity, config=args)
print(args)

In [ ]:
args = helper.args
device = helper.device
is_cuda = helper.device == torch.device('cuda')

In [ ]:
mean = (0.56019358, 0.52410121, 0.501457)
std = (0.23318603, 0.24300033, 0.24567522)
Dataset = getattr(import_module("dataset"), args.dataset)

In [ ]:
DataInfo = getattr(import_module("dataset"), "TrainInfo")
data_info = DataInfo(
    file_dir=None,
    data_dir=args.data_dir
)
train_df, valid_df, dist_df = data_info.split_dataset(args.val_ratio)

In [ ]:
# train_loader, valid_loader
def generate_loader(train_set, valid_set, transforms=None):
    train_batch_size = min(len(train_set), args.batch_size//20)
    valid_batch_size = min(len(valid_set), args.val_batch_size)

    transform_list = transforms if transforms else list(map(lambda trf: getattr(import_module("transform"), trf), args.transform))
    
    train_transform = transform_list[1](
        resize=args.resize,
        mean=train_set.mean,
        std=train_set.std,
    )
    valid_transform = transform_list[0](
        resize=args.resize,
        mean=train_set.mean,
        std=train_set.std,
    )

    train_set.set_transform(train_transform)
    valid_set.set_transform(valid_transform)

    train_loader = DataLoader(
        train_set,
        batch_size=train_batch_size,
        num_workers=mp.cpu_count()//2,
        shuffle=True,
        pin_memory=is_cuda,
        drop_last=True,
    )

    valid_loader = DataLoader(
        valid_set,
        batch_size=valid_batch_size,
        num_workers=mp.cpu_count()//2,
        shuffle=False,
        pin_memory=is_cuda,
        drop_last=True,
    )

    return train_loader, valid_loader


# model, criterion, optimizer, scheduler
def model_settings():
    Model = getattr(import_module("model"), args.model)
    model = Model(num_classes=num_classes, freeze=args.freeze).to(device)
    model = torch.nn.DataParallel(model)
    criterion = get_criterion(args.criterion)
    Optimizer = getattr(import_module('torch.optim'), args.optimizer)
    optimizer = Optimizer(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=args.lr,
        weight_decay=5e-4
    )
    scheduler = StepLR(optimizer, args.lr_decay_step, gamma=0.5)

    return model, criterion, optimizer, scheduler


# writer, save_dir
def other_settings(model_name=None):
    model_dir = '/opt/ml/image-classification-level1-05/model'
    save_dir = model_dir + '/' + model_name if model_name else helper.get_save_dir(dump=args.dump) 
    print(f"Save Directory : {save_dir}")
    writer = SummaryWriter(log_dir=save_dir)
    with open(os.path.join(save_dir, f'{args.mode}.json'), 'w', encoding='utf-8') as f:
        json.dump(vars(args), f, ensure_ascii=False, indent=4)

    return writer, save_dir

## 마스크 착용/미착용 데이터 분할
---

In [ ]:
train_mask = train_df.loc[train_df.Mask.isin(['Wear', 'Incorrect'])]
valid_mask = valid_df.loc[valid_df.Mask.isin(['Wear', 'Incorrect'])]

train_no_mask = train_df.loc[train_df.Mask.isin(['Not Wear'])]
valid_no_mask = valid_df.loc[valid_df.Mask.isin(['Not Wear'])]

# Mask 착용 -> Age 그룹 예측
---

## Generate Dataset
---

In [ ]:
train_set = Dataset(train_mask, mean=mean, std=std, label_col='ClassAge' + args.mode.capitalize())
valid_set = Dataset(valid_mask, mean=mean, std=std, label_col='ClassAge' + args.mode.capitalize())
num_classes = train_set.num_classes
print(num_classes)

## Settings for Train
---

In [ ]:
model_name = 'model_mask_age'

train_loader, valid_loader = generate_loader(train_set, valid_set)
model, criterion, optimizer, scheduler = model_settings()
writer, save_dir = other_settings(model_name)

## Train & Valid set evaluation
---

In [ ]:
# Initial Setting
best_model_acc = None
best_model_f1 = None
best_val_acc = 0
best_val_loss = np.inf
best_f1 = 0
val_labels = []
val_preds = []

for epoch in range(1, args.epochs + 1):
    loss_value = 0
    matches = 0
    accumulated_f1 = 0
    iter_count = 0

    for idx, (imgs, labels) in enumerate(train_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)

        outs = model(imgs)
        preds = torch.argmax(outs, dim=1)
        loss = criterion(outs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_value += loss.item()
        matches += (preds == labels).float().mean().item()
        accumulated_f1 += f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
        iter_count += 1

        if (idx + 1) % args.log_interval == 0:
            train_loss = loss_value / args.log_interval
            train_acc = matches / args.log_interval
            train_f1 = accumulated_f1 / iter_count
            current_lr = logger.get_lr(optimizer)

            writer.add_scalar("Train/loss", train_loss, epoch * len(train_loader) + idx)
            writer.add_scalar("Train/accuracy", train_acc, epoch * len(train_loader) + idx)
            writer.add_scalar("Train/f1", train_f1, epoch * len(train_loader) + idx)

            loss_value = 0
            matches = 0
    print(
        f'Epoch: {epoch:0{len(str(args.epochs))}d}/{args.epochs} '
        f'[{idx + 1:0{len(str(len(train_loader)))}d}/{len(train_loader)}]\n'
        f'\tTraining - accuracy: {train_acc:>3.2%}\tloss: {train_loss:>4.4f}\tf1: {train_f1:>4.4f}\tlr: {current_lr}'
    )

    scheduler.step()

    model.eval()
    with torch.no_grad():
        val_loss_items = []
        val_acc_items = []
        val_f1_items = []

        figure = None
        for val_batch in valid_loader:
            inputs, labels = val_batch
            # if epoch == args.epochs:
            val_labels.extend(map(torch.Tensor.item, labels))

            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            # if epoch == args.epochs:
            val_preds.extend(map(torch.Tensor.item, preds))

            loss_item = criterion(outs, labels).item()
            acc_item = (labels == preds).float().sum().item()
            f1_item = f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            val_f1_items.append(f1_item)


        val_loss = np.sum(val_loss_items) / len(valid_loader)
        val_acc = np.sum(val_acc_items) / len(valid_set)
        val_f1 = np.average(val_f1_items)
        best_val_loss = min(best_val_loss, val_loss)
        
        print(
            f'\tValidation - accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}'

            # f'Validation:\n'
            # f'accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}\n'
            # f'best acc : {best_val_acc:>3.2%}\tbest loss: {best_val_loss:>4.2f}\n'
        )
        if val_acc > best_val_acc:
            print(f"\tNew best model for val accuracy : {val_acc:3.2%}! saving the best model..")
            best_val_acc = val_acc
            best_model_acc = model
            
        if val_f1 > best_f1:
            print(f"\tNew best model for f1 : {val_f1:3.2f}! saving the best model..")
            best_f1 = val_f1
            best_model_f1 = model

        writer.add_scalar("Val/loss", val_loss, epoch)
        writer.add_scalar("Val/accuracy", val_acc, epoch)
        writer.add_scalar("Val/f1", val_f1, epoch)
        # writer.add_figure("results", figure, epoch)

        wandb.log({"Val/loss": val_loss, "Val/accuracy": val_acc, "Val/f1": val_f1})
    model.train()
    
torch.save(best_model_acc, os.path.join(save_dir, f'{args.mode if args.mode else model_name}.pt'))
torch.save(best_model_f1, os.path.join(save_dir, f'{args.mode if args.mode else model_name}_f1.pt'))
model_mask_age_acc = best_model_acc
model_mask_age_f1 = best_model_f1

# logger.save_confusion_matrix(num_classes=valid_set.num_classes, labels=val_labels, preds=val_preds, save_path=os.path.join(save_dir, 'confusion_matrix.png'))

<br><br><br><br>


# Mask 미착용 -> Age 그룹 예측
---

## Generate Dataset
---

In [ ]:
train_set = Dataset(train_no_mask, mean=mean, std=std, label_col='ClassAge' + args.mode.capitalize())
valid_set = Dataset(valid_no_mask, mean=mean, std=std, label_col='ClassAge' + args.mode.capitalize())
num_classes = train_set.num_classes
print(num_classes)

## Settings for Train
---

In [ ]:
model_name = 'model_nomask_age'

train_loader, valid_loader = generate_loader(train_set, valid_set)
model, criterion, optimizer, scheduler = model_settings()
writer, save_dir = other_settings(model_name)

## Train & Valid set evaluation
---

In [ ]:
# Initial Setting
best_model_acc = None
best_model_f1 = None
best_val_acc = 0
best_val_loss = np.inf
best_f1 = 0
val_labels = []
val_preds = []

for epoch in range(1, args.epochs + 1):
    loss_value = 0
    matches = 0
    accumulated_f1 = 0
    iter_count = 0

    for idx, (imgs, labels) in enumerate(train_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)

        outs = model(imgs)
        preds = torch.argmax(outs, dim=1)
        loss = criterion(outs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_value += loss.item()
        matches += (preds == labels).float().mean().item()
        accumulated_f1 += f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
        iter_count += 1

        if (idx + 1) % args.log_interval == 0:
            train_loss = loss_value / args.log_interval
            train_acc = matches / args.log_interval
            train_f1 = accumulated_f1 / iter_count
            current_lr = logger.get_lr(optimizer)

            writer.add_scalar("Train/loss", train_loss, epoch * len(train_loader) + idx)
            writer.add_scalar("Train/accuracy", train_acc, epoch * len(train_loader) + idx)
            writer.add_scalar("Train/f1", train_f1, epoch * len(train_loader) + idx)

            loss_value = 0
            matches = 0
    print(
        f'Epoch: {epoch:0{len(str(args.epochs))}d}/{args.epochs} '
        f'[{idx + 1:0{len(str(len(train_loader)))}d}/{len(train_loader)}]\n'
        f'\tTraining - accuracy: {train_acc:>3.2%}\tloss: {train_loss:>4.4f}\tf1: {train_f1:>4.4f}\tlr: {current_lr}'
    )

    scheduler.step()

    model.eval()
    with torch.no_grad():
        val_loss_items = []
        val_acc_items = []
        val_f1_items = []

        figure = None
        for val_batch in valid_loader:
            inputs, labels = val_batch
            # if epoch == args.epochs:
            val_labels.extend(map(torch.Tensor.item, labels))

            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            # if epoch == args.epochs:
            val_preds.extend(map(torch.Tensor.item, preds))

            loss_item = criterion(outs, labels).item()
            acc_item = (labels == preds).float().sum().item()
            f1_item = f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            val_f1_items.append(f1_item)


        val_loss = np.sum(val_loss_items) / len(valid_loader)
        val_acc = np.sum(val_acc_items) / len(valid_set)
        val_f1 = np.average(val_f1_items)
        best_val_loss = min(best_val_loss, val_loss)
        
        print(
            f'\tValidation - accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}'

            # f'Validation:\n'
            # f'accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}\n'
            # f'best acc : {best_val_acc:>3.2%}\tbest loss: {best_val_loss:>4.2f}\n'
        )
        if val_acc > best_val_acc:
            print(f"\tNew best model for val accuracy : {val_acc:3.2%}! saving the best model..")
            best_val_acc = val_acc
            best_model_acc = model
            
        if val_f1 > best_f1:
            print(f"\tNew best model for f1 : {val_f1:3.2f}! saving the best model..")
            best_f1 = val_f1
            best_model_f1 = model

        writer.add_scalar("Val/loss", val_loss, epoch)
        writer.add_scalar("Val/accuracy", val_acc, epoch)
        writer.add_scalar("Val/f1", val_f1, epoch)
        # writer.add_figure("results", figure, epoch)

        wandb.log({"Val/loss": val_loss, "Val/accuracy": val_acc, "Val/f1": val_f1})
    model.train()
    
torch.save(best_model_acc, os.path.join(save_dir, f'{args.mode if args.mode else model_name}.pt'))
torch.save(best_model_f1, os.path.join(save_dir, f'{args.mode if args.mode else model_name}_f1.pt'))
model_nomask_age_acc = best_model_acc
model_nomask_age_f1 = best_model_f1

# logger.save_confusion_matrix(num_classes=valid_set.num_classes, labels=val_labels, preds=val_preds, save_path=os.path.join(save_dir, 'confusion_matrix.png'))

<br><br><br><br>


# Mask 착용 -> Gender 그룹 예측
---

## Generate Dataset
---

In [ ]:
train_set = Dataset(train_mask, mean=mean, std=std, label_col='ClassGender' + args.mode.capitalize())
valid_set = Dataset(valid_mask, mean=mean, std=std, label_col='ClassGender' + args.mode.capitalize())
num_classes = train_set.num_classes
print(num_classes)

## Settings for Train
---

In [ ]:
model_name = 'model_mask_gender'

train_loader, valid_loader = generate_loader(train_set, valid_set)
model, criterion, optimizer, scheduler = model_settings()
writer, save_dir = other_settings(model_name)

## Train & Valid set evaluation
---

In [ ]:
# Initial Setting
best_model_acc = None
best_model_f1 = None
best_val_acc = 0
best_val_loss = np.inf
best_f1 = 0
val_labels = []
val_preds = []

for epoch in range(1, args.epochs + 1):
    loss_value = 0
    matches = 0
    accumulated_f1 = 0
    iter_count = 0

    for idx, (imgs, labels) in enumerate(train_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)

        outs = model(imgs)
        preds = torch.argmax(outs, dim=1)
        loss = criterion(outs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_value += loss.item()
        matches += (preds == labels).float().mean().item()
        accumulated_f1 += f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
        iter_count += 1

        if (idx + 1) % args.log_interval == 0:
            train_loss = loss_value / args.log_interval
            train_acc = matches / args.log_interval
            train_f1 = accumulated_f1 / iter_count
            current_lr = logger.get_lr(optimizer)

            writer.add_scalar("Train/loss", train_loss, epoch * len(train_loader) + idx)
            writer.add_scalar("Train/accuracy", train_acc, epoch * len(train_loader) + idx)
            writer.add_scalar("Train/f1", train_f1, epoch * len(train_loader) + idx)

            loss_value = 0
            matches = 0
    print(
        f'Epoch: {epoch:0{len(str(args.epochs))}d}/{args.epochs} '
        f'[{idx + 1:0{len(str(len(train_loader)))}d}/{len(train_loader)}]\n'
        f'\tTraining - accuracy: {train_acc:>3.2%}\tloss: {train_loss:>4.4f}\tf1: {train_f1:>4.4f}\tlr: {current_lr}'
    )

    scheduler.step()

    model.eval()
    with torch.no_grad():
        val_loss_items = []
        val_acc_items = []
        val_f1_items = []

        figure = None
        for val_batch in valid_loader:
            inputs, labels = val_batch
            # if epoch == args.epochs:
            val_labels.extend(map(torch.Tensor.item, labels))

            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            # if epoch == args.epochs:
            val_preds.extend(map(torch.Tensor.item, preds))

            loss_item = criterion(outs, labels).item()
            acc_item = (labels == preds).float().sum().item()
            f1_item = f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            val_f1_items.append(f1_item)


        val_loss = np.sum(val_loss_items) / len(valid_loader)
        val_acc = np.sum(val_acc_items) / len(valid_set)
        val_f1 = np.average(val_f1_items)
        best_val_loss = min(best_val_loss, val_loss)
        
        print(
            f'\tValidation - accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}'

            # f'Validation:\n'
            # f'accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}\n'
            # f'best acc : {best_val_acc:>3.2%}\tbest loss: {best_val_loss:>4.2f}\n'
        )
        if val_acc > best_val_acc:
            print(f"\tNew best model for val accuracy : {val_acc:3.2%}! saving the best model..")
            best_val_acc = val_acc
            best_model_acc = model
            
        if val_f1 > best_f1:
            print(f"\tNew best model for f1 : {val_f1:3.2f}! saving the best model..")
            best_f1 = val_f1
            best_model_f1 = model

        writer.add_scalar("Val/loss", val_loss, epoch)
        writer.add_scalar("Val/accuracy", val_acc, epoch)
        writer.add_scalar("Val/f1", val_f1, epoch)
        # writer.add_figure("results", figure, epoch)

        wandb.log({"Val/loss": val_loss, "Val/accuracy": val_acc, "Val/f1": val_f1})
    model.train()
    
torch.save(best_model_acc, os.path.join(save_dir, f'{args.mode if args.mode else model_name}.pt'))
torch.save(best_model_f1, os.path.join(save_dir, f'{args.mode if args.mode else model_name}_f1.pt'))
model_mask_gender_acc = best_model_acc
model_mask_gender_f1 = best_model_f1

# logger.save_confusion_matrix(num_classes=valid_set.num_classes, labels=val_labels, preds=val_preds, save_path=os.path.join(save_dir, 'confusion_matrix.png'))

<br><br><br><br>


# Mask 미착용 -> Gender 그룹 예측
---

## Generate Dataset
---

In [ ]:
train_set = Dataset(train_no_mask, mean=mean, std=std, label_col='ClassGender' + args.mode.capitalize())
valid_set = Dataset(valid_no_mask, mean=mean, std=std, label_col='ClassGender' + args.mode.capitalize())
num_classes = train_set.num_classes
print(num_classes)

## Settings for Train
---

In [ ]:
model_name = 'model_nomask_gender'

train_loader, valid_loader = generate_loader(train_set, valid_set)
model, criterion, optimizer, scheduler = model_settings()
writer, save_dir = other_settings(model_name)

## Train & Valid set evaluation
---

In [ ]:
# Initial Setting
best_model_acc = None
best_model_f1 = None
best_val_acc = 0
best_val_loss = np.inf
best_f1 = 0
val_labels = []
val_preds = []

for epoch in range(1, args.epochs + 1):
    loss_value = 0
    matches = 0
    accumulated_f1 = 0
    iter_count = 0

    for idx, (imgs, labels) in enumerate(train_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)

        outs = model(imgs)
        preds = torch.argmax(outs, dim=1)
        loss = criterion(outs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_value += loss.item()
        matches += (preds == labels).float().mean().item()
        accumulated_f1 += f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
        iter_count += 1

        if (idx + 1) % args.log_interval == 0:
            train_loss = loss_value / args.log_interval
            train_acc = matches / args.log_interval
            train_f1 = accumulated_f1 / iter_count
            current_lr = logger.get_lr(optimizer)

            writer.add_scalar("Train/loss", train_loss, epoch * len(train_loader) + idx)
            writer.add_scalar("Train/accuracy", train_acc, epoch * len(train_loader) + idx)
            writer.add_scalar("Train/f1", train_f1, epoch * len(train_loader) + idx)

            loss_value = 0
            matches = 0
    print(
        f'Epoch: {epoch:0{len(str(args.epochs))}d}/{args.epochs} '
        f'[{idx + 1:0{len(str(len(train_loader)))}d}/{len(train_loader)}]\n'
        f'\tTraining - accuracy: {train_acc:>3.2%}\tloss: {train_loss:>4.4f}\tf1: {train_f1:>4.4f}\tlr: {current_lr}'
    )

    scheduler.step()

    model.eval()
    with torch.no_grad():
        val_loss_items = []
        val_acc_items = []
        val_f1_items = []

        figure = None
        for val_batch in valid_loader:
            inputs, labels = val_batch
            # if epoch == args.epochs:
            val_labels.extend(map(torch.Tensor.item, labels))

            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            # if epoch == args.epochs:
            val_preds.extend(map(torch.Tensor.item, preds))

            loss_item = criterion(outs, labels).item()
            acc_item = (labels == preds).float().sum().item()
            f1_item = f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            val_f1_items.append(f1_item)


        val_loss = np.sum(val_loss_items) / len(valid_loader)
        val_acc = np.sum(val_acc_items) / len(valid_set)
        val_f1 = np.average(val_f1_items)
        best_val_loss = min(best_val_loss, val_loss)
        
        print(
            f'\tValidation - accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}'

            # f'Validation:\n'
            # f'accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}\n'
            # f'best acc : {best_val_acc:>3.2%}\tbest loss: {best_val_loss:>4.2f}\n'
        )
        if val_acc > best_val_acc:
            print(f"\tNew best model for val accuracy : {val_acc:3.2%}! saving the best model..")
            best_val_acc = val_acc
            best_model_acc = model
            
        if val_f1 > best_f1:
            print(f"\tNew best model for f1 : {val_f1:3.2f}! saving the best model..")
            best_f1 = val_f1
            best_model_f1 = model

        writer.add_scalar("Val/loss", val_loss, epoch)
        writer.add_scalar("Val/accuracy", val_acc, epoch)
        writer.add_scalar("Val/f1", val_f1, epoch)
        # writer.add_figure("results", figure, epoch)

        wandb.log({"Val/loss": val_loss, "Val/accuracy": val_acc, "Val/f1": val_f1})
    model.train()
    
torch.save(best_model_acc, os.path.join(save_dir, f'{args.mode if args.mode else model_name}.pt'))
torch.save(best_model_f1, os.path.join(save_dir, f'{args.mode if args.mode else model_name}_f1.pt'))
model_nomask_gender_acc = best_model_acc
model_nomask_gender_f1 = best_model_f1

# logger.save_confusion_matrix(num_classes=valid_set.num_classes, labels=val_labels, preds=val_preds, save_path=os.path.join(save_dir, 'confusion_matrix.png'))

<br><br><br><br>


# Mask 분류
---

## Generate Dataset
---

In [ ]:
train_set = Dataset(train_df, mean=mean, std=std, label_col='ClassMask')
valid_set = Dataset(valid_df, mean=mean, std=std, label_col='ClassMask')
num_classes = train_set.num_classes
print(num_classes)

## Settings for Train
---

In [ ]:
model_name = 'model_mask'

train_loader, valid_loader = generate_loader(train_set, valid_set)
model, criterion, optimizer, scheduler = model_settings()
writer, save_dir = other_settings(model_name)

## Train & Valid set evaluation
---

In [ ]:
# Initial Setting
best_model_acc = None
best_model_f1 = None
best_val_acc = 0
best_val_loss = np.inf
best_f1 = 0
val_labels = []
val_preds = []

for epoch in range(1, args.epochs + 1):
    loss_value = 0
    matches = 0
    accumulated_f1 = 0
    iter_count = 0

    for idx, (imgs, labels) in enumerate(train_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)

        outs = model(imgs)
        preds = torch.argmax(outs, dim=1)
        loss = criterion(outs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_value += loss.item()
        matches += (preds == labels).float().mean().item()
        accumulated_f1 += f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
        iter_count += 1

        if (idx + 1) % args.log_interval == 0:
            train_loss = loss_value / args.log_interval
            train_acc = matches / args.log_interval
            train_f1 = accumulated_f1 / iter_count
            current_lr = logger.get_lr(optimizer)

            writer.add_scalar("Train/loss", train_loss, epoch * len(train_loader) + idx)
            writer.add_scalar("Train/accuracy", train_acc, epoch * len(train_loader) + idx)
            writer.add_scalar("Train/f1", train_f1, epoch * len(train_loader) + idx)

            loss_value = 0
            matches = 0
    print(
        f'Epoch: {epoch:0{len(str(args.epochs))}d}/{args.epochs} '
        f'[{idx + 1:0{len(str(len(train_loader)))}d}/{len(train_loader)}]\n'
        f'\tTraining - accuracy: {train_acc:>3.2%}\tloss: {train_loss:>4.4f}\tf1: {train_f1:>4.4f}\tlr: {current_lr}'
    )

    scheduler.step()

    model.eval()
    with torch.no_grad():
        val_loss_items = []
        val_acc_items = []
        val_f1_items = []

        figure = None
        for val_batch in valid_loader:
            inputs, labels = val_batch
            # if epoch == args.epochs:
            val_labels.extend(map(torch.Tensor.item, labels))

            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            # if epoch == args.epochs:
            val_preds.extend(map(torch.Tensor.item, preds))

            loss_item = criterion(outs, labels).item()
            acc_item = (labels == preds).float().sum().item()
            f1_item = f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            val_f1_items.append(f1_item)


        val_loss = np.sum(val_loss_items) / len(valid_loader)
        val_acc = np.sum(val_acc_items) / len(valid_set)
        val_f1 = np.average(val_f1_items)
        best_val_loss = min(best_val_loss, val_loss)
        
        print(
            f'\tValidation - accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}'

            # f'Validation:\n'
            # f'accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}\n'
            # f'best acc : {best_val_acc:>3.2%}\tbest loss: {best_val_loss:>4.2f}\n'
        )
        if val_acc > best_val_acc:
            print(f"\tNew best model for val accuracy : {val_acc:3.2%}! saving the best model..")
            best_val_acc = val_acc
            best_model_acc = model
            
        if val_f1 > best_f1:
            print(f"\tNew best model for f1 : {val_f1:3.2f}! saving the best model..")
            best_f1 = val_f1
            best_model_f1 = model

        writer.add_scalar("Val/loss", val_loss, epoch)
        writer.add_scalar("Val/accuracy", val_acc, epoch)
        writer.add_scalar("Val/f1", val_f1, epoch)
        # writer.add_figure("results", figure, epoch)

        wandb.log({"Val/loss": val_loss, "Val/accuracy": val_acc, "Val/f1": val_f1})
    model.train()
    
torch.save(best_model_acc, os.path.join(save_dir, f'{args.mode if args.mode else model_name}.pt'))
torch.save(best_model_f1, os.path.join(save_dir, f'{args.mode if args.mode else model_name}_f1.pt'))
model_mask_acc = best_model_acc
model_mask_f1 = best_model_f1

# logger.save_confusion_matrix(num_classes=valid_set.num_classes, labels=val_labels, preds=val_preds, save_path=os.path.join(save_dir, 'confusion_matrix.png'))

<br><br><br><br>

# 모델 성능 체크
---

In [ ]:
def load_model(model_dir, device):
    # model_path = os.path.join(model_dir, args.model_name)
    model = torch.load(model_dir, map_location=device)
    return model 

In [ ]:
# Acc Models
model_mask_acc
model_mask_age_acc
model_mask_gender_acc

# F1 Models
model_mask_f1
model_mask_age_f1
model_mask_gender_f1

# model_mask = load_model(Path(args.model_dir).joinpath('mask_model4.pt'), device)

In [ ]:
train_set = Dataset(train_df, mean=mean, std=std, label_col='Class')
valid_set = Dataset(valid_df, mean=mean, std=std, label_col='Class')
num_classes = valid_set.num_classes
print(num_classes)


# Validation Set Transform
valid_transform = Transforms[0](
    resize=args.resize,
    mean=train_set.mean,
    std=train_set.std,
)
valid_set.set_transform(val_transform)

valid_loader = DataLoader(
    valid_set,
    batch_size=1,
    num_workers=mp.cpu_count() // 2,
    shuffle=False,
    pin_memory=is_cuda,
    drop_last=True,
)

In [ ]:
with torch.no_grad():
    val_loss_items = []
    val_acc_items = []
    val_f1_items = []

    for val_batch in tqdm(valid_loader):
        inputs, labels = val_batch

        if epoch == args.epochs:
            val_labels.extend(map(torch.Tensor.item, labels))

        inputs = inputs.to(device)
        labels = labels.to(device)

        outs = model_mask(inputs)
        preds_mask = torch.argmax(outs, dim=-1)

        # Mask Wear - 0: Wear, 1: Incorrect
        if int(preds_mask) in [0,1]:
            outs = model_mask_age_f1(inputs)
            preds_age = torch.argmax(outs, dim=-1)

            outs = model_mask_gender_f1(inputs)
            preds_gender = torch.argmax(outs, dim=-1)      

        # Mask Not Wear - 2: Not Wear
        elif int(preds_mask) in [2]:
            outs = model_nomask_age_f1(inputs)
            preds_age = torch.argmax(outs, dim=-1)

            outs = model_nomask_gender_f1(inputs)
            preds_gender = torch.argmax(outs, dim=-1)      

        else:
            print(preds_mask)
            raise ValueError

        preds = 6*preds_mask + 3*preds_gender + preds_age

        acc_item = (labels == preds).float().sum().item()
        f1_item = f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
        val_acc_items.append(acc_item)
        val_f1_items.append(f1_item)

        if figure is None:
            imgs = torch.clone(inputs).detach(
            ).cpu().permute(0, 2, 3, 1).numpy()
            imgs = train_set.denormalize_image(imgs, train_set.mean, train_set.std)
            figure = logger.grid_image(
                imgs=imgs, labels=labels, preds=preds,
                n=16, shuffle=args.dataset != "MaskSplitByProfileDataset"
            )

    val_loss = np.sum(val_loss_items) / len(valid_loader)
    val_acc = np.sum(val_acc_items) / len(valid_set)
    val_f1 = np.average(val_f1_items)
    best_val_loss = min(best_val_loss, val_loss)
    print(
        f'\tValidation - accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}'

        # f'Validation:\n'
        # f'accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}\n'
        # f'best acc : {best_val_acc:>3.2%}\tbest loss: {best_val_loss:>4.2f}\n'
    )

        # print(int(preds_mask), int(preds_gender), int(preds_age), '->', int(preds), '|', int(labels), '<-', int(labels)//6, int(labels)%6//3, int(labels)%6%3)


In [ ]:
    model.eval()
    with torch.no_grad():
        val_loss_items = []
        val_acc_items = []
        val_f1_items = []

        figure = None
        for val_batch in valid_loader:
            inputs, labels = val_batch
            if epoch == args.epochs:
                val_labels.extend(map(torch.Tensor.item, labels))

            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            if epoch == args.epochs:
                val_preds.extend(map(torch.Tensor.item, preds))

            loss_item = criterion(outs, labels).item()
            acc_item = (labels == preds).float().sum().item()
            f1_item = f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            val_f1_items.append(f1_item)

            if figure is None:
                imgs = torch.clone(inputs).detach(
                ).cpu().permute(0, 2, 3, 1).numpy()
                imgs = train_set.denormalize_image(imgs, train_set.mean, train_set.std)
                figure = logger.grid_image(
                    imgs=imgs, labels=labels, preds=preds,
                    n=16, shuffle=args.dataset != "MaskSplitByProfileDataset"
                )

        val_loss = np.sum(val_loss_items) / len(valid_loader)
        val_acc = np.sum(val_acc_items) / len(valid_set)
        val_f1 = np.average(val_f1_items)
        best_val_loss = min(best_val_loss, val_loss)
        print(
            f'\tValidation - accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}'

            # f'Validation:\n'
            # f'accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}\n'
            # f'best acc : {best_val_acc:>3.2%}\tbest loss: {best_val_loss:>4.2f}\n'
        )
        if val_acc > best_val_acc:
            print(f"\tNew best model for val accuracy : {val_acc:3.2%}! saving the best model..")
            torch.save(model, os.path.join(save_dir, f'{args.mode if args.mode else model_name}.pt'))
            best_val_acc = val_acc
            best_model_acc = model
            
        if val_f1 > best_f1:
            print(f"\tNew best model for f1 : {val_f1:3.2f}! saving the best model..")
            torch.save(model, os.path.join(save_dir, f'{args.mode if args.mode else model_name}_f1.pt'))
            best_f1 = val_f1
            best_model_f1 = model

        writer.add_scalar("Val/loss", val_loss, epoch)
        writer.add_scalar("Val/accuracy", val_acc, epoch)
        writer.add_scalar("Val/f1", val_f1, epoch)
        writer.add_figure("results", figure, epoch)
    model.train()
    print('')
    

model_mask_acc = best_model_acc
model_mask_f1 = best_model_f1



logger.save_confusion_matrix(num_classes=valid_set.num_classes, labels=val_labels, preds=val_preds, save_path=os.path.join(save_dir, 'confusion_matrix.png'))


In [ ]:
def _show(tensor_image):
    tensor_image = (tensor_image).to('cpu')

    tensor_image = tensor_image.view(tensor_image.shape[1], tensor_image.shape[2], tensor_image.shape[0])
    print(type(tensor_image), tensor_image.shape)

    plt.imshow(tensor_image)
    plt.show()
# model_mask_acc(inputs)

### 마스크 착용 - 성별 그룹에 대한 훈련/검증 데이터셋 분할

In [ ]:
train_set = Dataset(train_mask, mean=mean, std=std, label_col='ClassGender' + args.mode.capitalize())
valid_set = Dataset(valid_mask, mean=mean, std=std, label_col='ClassGender' + args.mode.capitalize())
num_classes = valid_set.num_classes

### Transform 및 data_loader


In [ ]:
Transforms = list(map(lambda trf: getattr(import_module("transform"), trf), args.transform))
val_transform = Transforms[0](
    resize=args.resize,
    mean=train_set.mean,
    std=train_set.std,
)
train_transform = Transforms[1](
    resize=args.resize,
    mean=train_set.mean,
    std=train_set.std,
)
train_set.set_transform(train_transform)
valid_set.set_transform(val_transform)

train_loader = DataLoader(
    train_set,
    batch_size=args.batch_size,
    num_workers=mp.cpu_count() // 2,
    shuffle=True,
    pin_memory=is_cuda,
    drop_last=True,
)

valid_loader = DataLoader(
    valid_set,
    batch_size=args.val_batch_size,
    num_workers=mp.cpu_count() // 2,
    shuffle=False,
    pin_memory=is_cuda,
    drop_last=True,
)

### 모델 설정

In [ ]:
Model = getattr(import_module("model"), args.model)
model = Model(num_classes=num_classes, freeze=args.freeze).to(device)
model = torch.nn.DataParallel(model)
criterion = get_criterion(args.criterion)
Optimizer = getattr(import_module('torch.optim'), args.optimizer)
optimizer = Optimizer(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=args.lr,
    weight_decay=5e-4
)
scheduler = StepLR(optimizer, args.lr_decay_step, gamma=0.5)

### 기타 설정

In [ ]:
save_dir = helper.get_save_dir(dump=args.dump)
writer = SummaryWriter(log_dir=save_dir)
with open(os.path.join(save_dir, f'{args.mode}.json'), 'w', encoding='utf-8') as f:
    json.dump(vars(args), f, ensure_ascii=False, indent=4)

# Initial Setting
best_val_acc = 0
best_val_loss = np.inf
best_f1 = 0

val_labels = []
val_preds = []

In [ ]:
model_name = 'model_mask_to_age'

for epoch in range(1, args.epochs + 1):
    loss_value = 0
    matches = 0
    accumulated_f1 = 0
    iter_count = 0

    for idx, (imgs, labels) in enumerate(train_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)

        outs = model(imgs)
        preds = torch.argmax(outs, dim=1)
        loss = criterion(outs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_value += loss.item()
        matches += (preds == labels).float().mean().item()
        accumulated_f1 += f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
        iter_count += 1

        if (idx + 1) % args.log_interval == 0:
            train_loss = loss_value / args.log_interval
            train_acc = matches / args.log_interval
            train_f1 = accumulated_f1 / iter_count
            current_lr = logger.get_lr(optimizer)

            writer.add_scalar("Train/loss", train_loss, epoch * len(train_loader) + idx)
            writer.add_scalar("Train/accuracy", train_acc, epoch * len(train_loader) + idx)
            writer.add_scalar("Train/f1", train_f1, epoch * len(train_loader) + idx)

            loss_value = 0
            matches = 0
    print(
        f'Epoch: {epoch:0{len(str(args.epochs))}d}/{args.epochs} '
        f'[{idx + 1:0{len(str(len(train_loader)))}d}/{len(train_loader)}]\n'
        f'\tTraining - accuracy: {train_acc:>3.2%}\tloss: {train_loss:>4.4f}\tf1: {train_f1:>4.4f}\tlr: {current_lr}'
    )

    scheduler.step()

    model.eval()
    with torch.no_grad():
        val_loss_items = []
        val_acc_items = []
        val_f1_items = []

        figure = None
        for val_batch in valid_loader:
            inputs, labels = val_batch
            if epoch == args.epochs:
                val_labels.extend(map(torch.Tensor.item, labels))

            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            if epoch == args.epochs:
                val_preds.extend(map(torch.Tensor.item, preds))

            loss_item = criterion(outs, labels).item()
            acc_item = (labels == preds).float().sum().item()
            f1_item = f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            val_f1_items.append(f1_item)

            if figure is None:
                imgs = torch.clone(inputs).detach(
                ).cpu().permute(0, 2, 3, 1).numpy()
                imgs = train_set.denormalize_image(imgs, train_set.mean, train_set.std)
                figure = logger.grid_image(
                    imgs=imgs, labels=labels, preds=preds,
                    n=16, shuffle=args.dataset != "MaskSplitByProfileDataset"
                )

        val_loss = np.sum(val_loss_items) / len(valid_loader)
        val_acc = np.sum(val_acc_items) / len(valid_set)
        val_f1 = np.average(val_f1_items)
        best_val_loss = min(best_val_loss, val_loss)
        print(
            f'\tValidation - accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}'

            # f'Validation:\n'
            # f'accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}\n'
            # f'best acc : {best_val_acc:>3.2%}\tbest loss: {best_val_loss:>4.2f}\n'
        )
        if val_acc > best_val_acc:
            print(f"\tNew best model for val accuracy : {val_acc:3.2%}! saving the best model..")
            torch.save(model, os.path.join(save_dir, f'{args.mode if args.mode else model_name}.pt'))
            best_val_acc = val_acc
        if val_f1 > best_f1:
            print(f"\tNew best model for f1 : {val_f1:3.2f}! saving the best model..")
            torch.save(model, os.path.join(save_dir, f'{args.mode if args.mode else model_name}f1.pt'))
            best_f1 = val_f1
        writer.add_scalar("Val/loss", val_loss, epoch)
        writer.add_scalar("Val/accuracy", val_acc, epoch)
        writer.add_scalar("Val/f1", val_f1, epoch)
        writer.add_figure("results", figure, epoch)
    model.train()
    print('')
    



logger.save_confusion_matrix(num_classes=valid_set.num_classes, labels=val_labels, preds=val_preds, save_path=os.path.join(save_dir, 'confusion_matrix.png'))



## Mask 탐지
---

### 마스크 착용 / 미착용 데이터

In [ ]:
train_mask = train_df.loc[train_df.Mask.isin(['Wear', 'Incorrect'])]
valid_mask = valid_df.loc[valid_df.Mask.isin(['Wear', 'Incorrect'])]

train_no_mask = train_df.loc[train_df.Mask.isin(['Not Wear'])]
valid_no_mask = valid_df.loc[valid_df.Mask.isin(['Not Wear'])]

### 마스크 착용/미착용에 대한 성별 데이터셋

In [ ]:
train_set = Dataset(train_mask, mean=mean, std=std, label_col='ClassGender' + args.mode.capitalize())
valid_set = Dataset(valid_mask, mean=mean, std=std, label_col='ClassGender' + args.mode.capitalize())
num_classes = valid_set.num_classes

### Transform 및 data_loader


In [ ]:
Transforms = list(map(lambda trf: getattr(import_module("transform"), trf), args.transform))
val_transform = Transforms[0](
    resize=args.resize,
    mean=train_set.mean,
    std=train_set.std,
)
train_transform = Transforms[1](
    resize=args.resize,
    mean=train_set.mean,
    std=train_set.std,
)
train_set.set_transform(train_transform)
valid_set.set_transform(val_transform)

train_loader = DataLoader(
    train_set,
    batch_size=args.batch_size,
    num_workers=mp.cpu_count() // 2,
    shuffle=True,
    pin_memory=is_cuda,
    drop_last=True,
)

valid_loader = DataLoader(
    valid_set,
    batch_size=args.val_batch_size,
    num_workers=mp.cpu_count() // 2,
    shuffle=False,
    pin_memory=is_cuda,
    drop_last=True,
)

### 모델 설정

In [ ]:
Model = getattr(import_module("model"), args.model)
model = Model(num_classes=num_classes, freeze=args.freeze).to(device)
model = torch.nn.DataParallel(model)
criterion = get_criterion(args.criterion)
Optimizer = getattr(import_module('torch.optim'), args.optimizer)
optimizer = Optimizer(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=args.lr,
    weight_decay=5e-4
)
scheduler = StepLR(optimizer, args.lr_decay_step, gamma=0.5)

### 기타 설정

In [ ]:
save_dir = helper.get_save_dir(dump=args.dump)
writer = SummaryWriter(log_dir=save_dir)
with open(os.path.join(save_dir, f'{args.mode}.json'), 'w', encoding='utf-8') as f:
    json.dump(vars(args), f, ensure_ascii=False, indent=4)

# Initial Setting
best_val_acc = 0
best_val_loss = np.inf
best_f1 = 0

val_labels = []
val_preds = []

In [ ]:
for epoch in range(1, args.epochs + 1):
    loss_value = 0
    matches = 0
    accumulated_f1 = 0
    iter_count = 0

    for idx, (imgs, labels) in enumerate(train_loader):
        imgs = imgs.to(device)
        labels = labels.to(device)

        outs = model(imgs)
        preds = torch.argmax(outs, dim=1)
        loss = criterion(outs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_value += loss.item()
        matches += (preds == labels).float().mean().item()
        accumulated_f1 += f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
        iter_count += 1

        if (idx + 1) % args.log_interval == 0:
            train_loss = loss_value / args.log_interval
            train_acc = matches / args.log_interval
            train_f1 = accumulated_f1 / iter_count
            current_lr = logger.get_lr(optimizer)

            writer.add_scalar("Train/loss", train_loss, epoch * len(train_loader) + idx)
            writer.add_scalar("Train/accuracy", train_acc, epoch * len(train_loader) + idx)
            writer.add_scalar("Train/f1", train_f1, epoch * len(train_loader) + idx)

            loss_value = 0
            matches = 0
    print(
        f'Epoch: {epoch:0{len(str(args.epochs))}d}/{args.epochs} '
        f'[{idx + 1:0{len(str(len(train_loader)))}d}/{len(train_loader)}]\n'
        f'\tTraining - accuracy: {train_acc:>3.2%}\tloss: {train_loss:>4.4f}\tf1: {train_f1:>4.4f}\tlr: {current_lr}'
    )

    scheduler.step()

    model.eval()
    with torch.no_grad():
        val_loss_items = []
        val_acc_items = []
        val_f1_items = []

        figure = None
        for val_batch in valid_loader:
            inputs, labels = val_batch
            if epoch == args.epochs:
                val_labels.extend(map(torch.Tensor.item, labels))

            inputs = inputs.to(device)
            labels = labels.to(device)

            outs = model(inputs)
            preds = torch.argmax(outs, dim=-1)
            if epoch == args.epochs:
                val_preds.extend(map(torch.Tensor.item, preds))

            loss_item = criterion(outs, labels).item()
            acc_item = (labels == preds).float().sum().item()
            f1_item = f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
            val_loss_items.append(loss_item)
            val_acc_items.append(acc_item)
            val_f1_items.append(f1_item)

            if figure is None:
                imgs = torch.clone(inputs).detach(
                ).cpu().permute(0, 2, 3, 1).numpy()
                imgs = train_set.denormalize_image(imgs, train_set.mean, train_set.std)
                figure = logger.grid_image(
                    imgs=imgs, labels=labels, preds=preds,
                    n=16, shuffle=args.dataset != "MaskSplitByProfileDataset"
                )

        val_loss = np.sum(val_loss_items) / len(valid_loader)
        val_acc = np.sum(val_acc_items) / len(valid_set)
        val_f1 = np.average(val_f1_items)
        best_val_loss = min(best_val_loss, val_loss)
        print(
            f'\tValidation - accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}'

            # f'Validation:\n'
            # f'accuracy: {val_acc:>3.2%}\tloss: {val_loss:>4.2f}\tf1: {val_f1:>4.2f}\n'
            # f'best acc : {best_val_acc:>3.2%}\tbest loss: {best_val_loss:>4.2f}\n'
        )
        if val_acc > best_val_acc:
            print(f"\tNew best model for val accuracy : {val_acc:3.2%}! saving the best model..")
            torch.save(model, os.path.join(save_dir, f'{args.mode if args.mode else args.model_name}.pt'))
            best_val_acc = val_acc
        if val_f1 > best_f1:
            print(f"\tNew best model for f1 : {val_f1:3.2f}! saving the best model..")
            torch.save(model, os.path.join(save_dir, f'{args.mode if args.mode else args.model_name}f1.pt'))
            best_f1 = val_f1
        writer.add_scalar("Val/loss", val_loss, epoch)
        writer.add_scalar("Val/accuracy", val_acc, epoch)
        writer.add_scalar("Val/f1", val_f1, epoch)
        writer.add_figure("results", figure, epoch)
    model.train()
    print('')
    



logger.save_confusion_matrix(num_classes=valid_set.num_classes, labels=val_labels, preds=val_preds, save_path=os.path.join(save_dir, 'confusion_matrix.png'))

